In [1]:
# | default_exp core.guardrails

In [2]:
# | export
from typing import Dict, Any, List, Optional, Callable, Union
from dataclasses import dataclass
from enum import Enum
import re
import json


class GuardrailType(Enum):
    INPUT_VALIDATION = "input_validation"
    OUTPUT_FILTERING = "output_filtering" 
    TOOL_RESTRICTION = "tool_restriction"
    CONTENT_SAFETY = "content_safety"
    RATE_LIMITING = "rate_limiting"


class GuardrailAction(Enum):
    ALLOW = "allow"
    BLOCK = "block"
    MODIFY = "modify"
    WARN = "warn"


@dataclass
class GuardrailResult:
    """Result of guardrail evaluation"""
    action: GuardrailAction
    passed: bool
    message: Optional[str] = None
    modified_content: Optional[str] = None
    metadata: Optional[Dict[str, Any]] = None


class Guardrail:
    """Base class for all guardrails"""
    
    def __init__(self, name: str, guardrail_type: GuardrailType):
        self.name = name
        self.type = guardrail_type
        self.enabled = True
    
    def evaluate(self, content: str, context: Optional[Dict[str, Any]] = None) -> GuardrailResult:
        """Evaluate content against this guardrail"""
        raise NotImplementedError
    
    def enable(self):
        """Enable this guardrail"""
        self.enabled = True
    
    def disable(self):
        """Disable this guardrail"""
        self.enabled = False


class ContentSafetyGuardrail(Guardrail):
    """Guardrail for content safety"""
    
    def __init__(self):
        super().__init__("content_safety", GuardrailType.CONTENT_SAFETY)
        self.blocked_patterns = [
            r'\b(password|secret|api[_-]?key|token)\s*[:=]\s*\S+',
            r'\b\d{4}[-\s]?\d{4}[-\s]?\d{4}[-\s]?\d{4}\b',  # Credit card
            r'\b\d{3}-\d{2}-\d{4}\b',  # SSN
            r'rm\s+-rf\s+/',  # Dangerous rm commands
            r'sudo\s+rm',  # Sudo rm commands
        ]
        self.warning_patterns = [
            r'\bsudo\b',
            r'\bchmod\s+777\b',
            r'\bdd\s+if=',
        ]
    
    def evaluate(self, content: str, context: Optional[Dict[str, Any]] = None) -> GuardrailResult:
        if not self.enabled:
            return GuardrailResult(GuardrailAction.ALLOW, True)
        
        # Check for blocked patterns
        for pattern in self.blocked_patterns:
            if re.search(pattern, content, re.IGNORECASE):
                return GuardrailResult(
                    GuardrailAction.BLOCK,
                    False,
                    f"Content blocked: Contains sensitive information or dangerous command"
                )
        
        # Check for warning patterns
        for pattern in self.warning_patterns:
            if re.search(pattern, content, re.IGNORECASE):
                return GuardrailResult(
                    GuardrailAction.WARN,
                    True,
                    f"Warning: Content contains potentially dangerous command"
                )
        
        return GuardrailResult(GuardrailAction.ALLOW, True)


class ToolRestrictionGuardrail(Guardrail):
    """Guardrail for restricting tool usage"""
    
    def __init__(self, allowed_tools: Optional[List[str]] = None, blocked_tools: Optional[List[str]] = None):
        super().__init__("tool_restriction", GuardrailType.TOOL_RESTRICTION)
        self.allowed_tools = allowed_tools
        self.blocked_tools = blocked_tools or []
    
    def evaluate(self, content: str, context: Optional[Dict[str, Any]] = None) -> GuardrailResult:
        if not self.enabled:
            return GuardrailResult(GuardrailAction.ALLOW, True)
        
        # Extract tool calls from context
        tool_calls = context.get("tool_calls", []) if context else []
        
        for tool_call in tool_calls:
            tool_name = tool_call.get("function", {}).get("name")
            
            if tool_name in self.blocked_tools:
                return GuardrailResult(
                    GuardrailAction.BLOCK,
                    False,
                    f"Tool '{tool_name}' is blocked by policy"
                )
            
            if self.allowed_tools and tool_name not in self.allowed_tools:
                return GuardrailResult(
                    GuardrailAction.BLOCK,
                    False,
                    f"Tool '{tool_name}' is not in allowed tools list"
                )
        
        return GuardrailResult(GuardrailAction.ALLOW, True)


class InputValidationGuardrail(Guardrail):
    """Guardrail for input validation"""
    
    def __init__(self, max_length: int = 10000, min_length: int = 1):
        super().__init__("input_validation", GuardrailType.INPUT_VALIDATION)
        self.max_length = max_length
        self.min_length = min_length
        self.blocked_keywords = ["hack", "exploit", "bypass", "jailbreak"]
    
    def evaluate(self, content: str, context: Optional[Dict[str, Any]] = None) -> GuardrailResult:
        if not self.enabled:
            return GuardrailResult(GuardrailAction.ALLOW, True)
        
        # Length validation
        if len(content) > self.max_length:
            return GuardrailResult(
                GuardrailAction.BLOCK,
                False,
                f"Input too long: {len(content)} > {self.max_length} characters"
            )
        
        if len(content) < self.min_length:
            return GuardrailResult(
                GuardrailAction.BLOCK,
                False,
                f"Input too short: {len(content)} < {self.min_length} characters"
            )
        
        # Keyword validation
        content_lower = content.lower()
        for keyword in self.blocked_keywords:
            if keyword in content_lower:
                return GuardrailResult(
                    GuardrailAction.WARN,
                    True,
                    f"Warning: Input contains potentially problematic keyword: {keyword}"
                )
        
        return GuardrailResult(GuardrailAction.ALLOW, True)


class OutputFilteringGuardrail(Guardrail):
    """Guardrail for filtering output content"""
    
    def __init__(self):
        super().__init__("output_filtering", GuardrailType.OUTPUT_FILTERING)
        self.sensitive_patterns = [
            r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',  # Email
            r'\b\d{3}-\d{3}-\d{4}\b',  # Phone number
            r'\b(?:\d{1,3}\.){3}\d{1,3}\b',  # IP address
        ]
    
    def evaluate(self, content: str, context: Optional[Dict[str, Any]] = None) -> GuardrailResult:
        if not self.enabled:
            return GuardrailResult(GuardrailAction.ALLOW, True)
        
        modified_content = content
        modifications_made = False
        
        for pattern in self.sensitive_patterns:
            matches = re.findall(pattern, content)
            if matches:
                for match in matches:
                    # Replace with redacted version
                    redacted = "[REDACTED]"
                    modified_content = modified_content.replace(match, redacted)
                    modifications_made = True
        
        if modifications_made:
            return GuardrailResult(
                GuardrailAction.MODIFY,
                True,
                "Output modified: Sensitive information redacted",
                modified_content
            )
        
        return GuardrailResult(GuardrailAction.ALLOW, True)


class RateLimitingGuardrail(Guardrail):
    """Guardrail for rate limiting"""
    
    def __init__(self, max_requests_per_minute: int = 60):
        super().__init__("rate_limiting", GuardrailType.RATE_LIMITING)
        self.max_requests_per_minute = max_requests_per_minute
        self.request_history: List[float] = []
    
    def evaluate(self, content: str, context: Optional[Dict[str, Any]] = None) -> GuardrailResult:
        if not self.enabled:
            return GuardrailResult(GuardrailAction.ALLOW, True)
        
        import time
        current_time = time.time()
        
        # Clean old requests (older than 1 minute)
        self.request_history = [t for t in self.request_history if current_time - t < 60]
        
        # Check rate limit
        if len(self.request_history) >= self.max_requests_per_minute:
            return GuardrailResult(
                GuardrailAction.BLOCK,
                False,
                f"Rate limit exceeded: {len(self.request_history)} requests in last minute"
            )
        
        # Add current request
        self.request_history.append(current_time)
        
        return GuardrailResult(GuardrailAction.ALLOW, True)


class GuardrailManager:
    """Manages multiple guardrails"""
    
    def __init__(self):
        self.guardrails: List[Guardrail] = []
        self.enforcement_mode = "strict"  # strict, permissive, logging_only
    
    def add_guardrail(self, guardrail: Guardrail):
        """Add a guardrail"""
        self.guardrails.append(guardrail)
    
    def remove_guardrail(self, name: str):
        """Remove a guardrail by name"""
        self.guardrails = [g for g in self.guardrails if g.name != name]
    
    def evaluate_input(self, content: str, context: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """Evaluate input against all guardrails"""
        results = []
        overall_passed = True
        final_action = GuardrailAction.ALLOW
        
        for guardrail in self.guardrails:
            if guardrail.type in [GuardrailType.INPUT_VALIDATION, GuardrailType.CONTENT_SAFETY]:
                result = guardrail.evaluate(content, context)
                results.append({
                    "guardrail": guardrail.name,
                    "result": result
                })
                
                if not result.passed:
                    overall_passed = False
                
                if result.action == GuardrailAction.BLOCK:
                    final_action = GuardrailAction.BLOCK
        
        return {
            "passed": overall_passed,
            "action": final_action,
            "results": results,
            "enforcement_mode": self.enforcement_mode
        }
    
    def evaluate_output(self, content: str, context: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """Evaluate output against all guardrails"""
        results = []
        modified_content = content
        
        for guardrail in self.guardrails:
            if guardrail.type == GuardrailType.OUTPUT_FILTERING:
                result = guardrail.evaluate(modified_content, context)
                results.append({
                    "guardrail": guardrail.name,
                    "result": result
                })
                
                if result.modified_content:
                    modified_content = result.modified_content
        
        return {
            "original_content": content,
            "modified_content": modified_content,
            "results": results
        }
    
    def set_enforcement_mode(self, mode: str):
        """Set enforcement mode: strict, permissive, logging_only"""
        self.enforcement_mode = mode


def create_default_guardrails() -> GuardrailManager:
    """Create a default set of guardrails"""
    manager = GuardrailManager()
    
    # Add default guardrails
    manager.add_guardrail(ContentSafetyGuardrail())
    manager.add_guardrail(InputValidationGuardrail())
    manager.add_guardrail(OutputFilteringGuardrail())
    manager.add_guardrail(RateLimitingGuardrail())
    
    # Add tool restrictions for sensitive tools
    dangerous_tools = ["execute_bash", "fs_write"]
    manager.add_guardrail(ToolRestrictionGuardrail(blocked_tools=dangerous_tools))
    
    return manager
